## Towards the comparison of the signal-to-noise achieved with different filter widths

## Signal-to-noise calculations

As a function of exposure time, what is the signal-to-noise for the different filter widths?

How does that change when the CWL of the filters changes?

What is the benefit of the narrow bandwidth filters over larger bandwidth?


### Some background

The SNR is calculated following the equation (see, e.g., pg 407 of "Spectroscopic Instrumentation" by Eversberg and Vollmann): 

$SNR = \frac{I t}{\sqrt{I t + B t + D t + R^2}}$, 

where I is the signal over some number of pixels, B is the sky background over some number of pixels, D is the dark current over some number of pixels and R is the readout noise of the detector.  The exposure time, t, is usually given in seconds.

In [4]:
def Dragonfly():
    number_of_lenses = 48.
    area_lens = np.pi*(14.3/2)**2 * number_of_lenses * u.cm**2               # 48 * 14.3 cm diameter lenses
    pix_size = 2.8                                              # arcsec
    ang_size_pixel  = (pix_size * (1./206265.))**2 * u.rad**2   # rad^2, the pixel size of the CCD

    tau_l = 0.85  # transmittance of the Dragonfly lens
    QE = 0.48     # quantum efficiency of the CCDs
    
    tau_f = 1.    # transmittance of the Halpha filter -- assumed for now

    # The following are per pixel
    D = 0.04 *u.photon/u.second                             # dark current (electrons / s) -- PER PIXEL??
    R_squared = 10.**2 * u.photon                           # read noise (electrons)

    return [area_lens, ang_size_pixel, pix_size, tau_l, tau_f, QE, D, R_squared]

In [2]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
from astropy import constants as const
from astropy import units as u
from astropy.io import fits

In [16]:
[area_lens, ang_size_pixel, pix_size, tau_l, tau_f, QE, D, R_squared] = Dragonfly()

B = 0.52135 *u.photon/u.second/u.nm/u.arcsec**2/u.m**2  

# centred at 656.3 nm
B_3nm = 1.52495*u.photon/u.second/u.arcsec**2/u.m**2  # already times the filter width
B_08nm = 0.383995*u.photon/u.second/u.arcsec**2/u.m**2  # already times the filter width

# centred at 659.9 nm
B_3nm = 1.387259*u.photon/u.second/u.arcsec**2/u.m**2  # already times the filter width
B_08nm = 0.378483*u.photon/u.second/u.arcsec**2/u.m**2  # already times the filter width


#SB_ph_one = 10**-2. * u.photon / (u.cm**2 * u.rad**2 * u.second)
#SB_ph_two = 10**-1. * u.photon / (u.cm**2 * u.rad**2 * u.second)
SB_ph_zero = 10**0. * u.photon / (u.cm**2 * u.rad**2 * u.second)
SB_ph_one = 10**1. * u.photon / (u.cm**2 * u.rad**2 * u.second)
SB_ph_two = 10**2. * u.photon / (u.cm**2 * u.rad**2 * u.second)
SB_ph_three = 10**3. * u.photon / (u.cm**2 * u.rad**2 * u.second)

numpixel = 100.0 * 100.0/(pix_size**2)   # 100-arcsec-a-side pixels
#numpixel = 10.0 * 10.0/(pix_size**2)   # 10-arcsec-a-side pixels

def calcSNR(SB_ph,B,time):
    S = SB_ph * QE * tau_l * tau_f * area_lens * ang_size_pixel
    signal = S*time*numpixel
    B_tot = B * QE * tau_l * tau_f * area_lens.to(u.m**2) * ang_size_pixel.to(u.arcsec**2)
    print('total S: %s'%S)
    print('total B: %s'%B_tot)
    sigma = np.sqrt(signal + B_tot*time*numpixel + D*time*numpixel + R_squared*numpixel)
    SNR = signal/sigma
    return SNR

time = 60.*60.*60. * u.second
SNR = calcSNR(SB_ph_three,B_3nm,time)
print SNR
print ""
time = 60.*60.*60. * u.second
SNR = calcSNR(SB_ph_three,B_08nm,time)
print SNR

total S: 0.000579599947142 ph / s
total B: 3.4208667244 ph / s
5.17058092296 ph(1/2)

total S: 0.000579599947142 ph / s
total B: 0.933307983911 ph / s
9.74629527788 ph(1/2)


In [ ]:
# pick how long to calculate SNR over
time = np.arange(1,60.*60.*1000000.,1000)* u.second
time = np.arange(1,60.*60.*280.)* u.second
limit = time[-1]/u.second

for SB_ph in [SB_ph_1]:
    ## Signal
    S = SB_ph * QE * tau_l * tau_f * area_lens * ang_size_pixel
    signal = S*time*numpixel
    ## Noise
    B_tot = B * QE * tau_l * tau_f * 3.*u.nm * area_lens.to(u.m**2) * ang_size_pixel.to(u.arcsec**2)
    sigma = np.sqrt(signal + B_tot*time*numpixel + D*time*numpixel + R_squared*numpixel)

    # Note that the signal is about an eighth the strength of the background sky signal: IS THIS TRUE HERE?
    print('Halpha Signal: '+str(S))
    print('Background Sky: ' +str(B_tot))

    plt.plot(time/3600., signal/sigma, label=str(SB_ph))#,label= 'num pixels '+str(round(numpixel)))

plt.plot([0,limit/3600.],[5,5],'k--')   
plt.title(r'H$\alpha$ Halo Emission')
plt.legend(title='Surface Brightness:')
plt.xlim(0,limit/3600.)
plt.ylim(0,10)
plt.xlabel('time (hours)')
plt.ylabel('SNR')

plt.plot([4.,4.],[0,10],'k--')  # ~4 hour ~250min
plt.plot([70,70],[0,10],'k--')  # ~70 hour ~4200min

